In [ ]:
# Importar as bibliotecas
import pandas as pd
import random

# Ler os dataframes
df_mensagens = pd.read_csv("mensagens.csv")
df_categorias = pd.read_csv("categorias.csv")

# Definir o tamanho da amostra
n = 100

# Selecionar n mensagens aleatoriamente
amostra = df_mensagens.sample(n)

# Criar uma lista vazia para armazenar as categorias
rotulos = []

# Para cada mensagem na amostra
for i in range(n):
  # Exibir a mensagem
  print(amostra.iloc[i]["mensagem"])
  # Exibir as categorias e subcategorias possíveis
  print(df_categorias)
  # Pedir ao usuário para digitar o código da categoria e subcategoria correspondente
  codigo = input("Digite o código da categoria e subcategoria separados por um ponto (ex: 1.2): ")
  # Adicionar o código à lista de rótulos
  rotulos.append(codigo)

# Adicionar a lista de rótulos como uma nova coluna no dataframe da amostra
amostra["rotulo"] = rotulos

# Salvar o dataframe da amostra como um novo arquivo csv
amostra.to_csv("amostra_rotulada.csv", index=False)

In [ ]:
# Importar as bibliotecas
import pandas as pd
import spacy
from spacy.matcher import Matcher

# Carregar o modelo de português do spaCy
nlp = spacy.load("pt_core_news_sm")

# Criar um objeto Matcher
matcher = Matcher(nlp.vocab)

# Ler os dataframes
df_mensagens = pd.read_csv("mensagens.csv")
df_categorias = pd.read_csv("categorias.csv")

# Para cada categoria e subcategoria no dataframe de categorias
for i in range(len(df_categorias)):
  # Obter o código, o nome e as palavras-chave da categoria e subcategoria
  codigo = df_categorias.iloc[i]["codigo"]
  nome = df_categorias.iloc[i]["nome"]
  palavras_chave = df_categorias.iloc[i]["palavras_chave"]
  # Transformar as palavras-chave em uma lista de strings
  palavras_chave = palavras_chave.split(",")
  # Criar uma lista vazia para armazenar os padrões de busca
  padroes = []
  # Para cada palavra-chave na lista
  for palavra in palavras_chave:
    # Criar um padrão de busca que corresponda à palavra-chave como um token inteiro, ignorando o caso
    padrao = [{"LOWER": palavra.lower()}]
    # Adicionar o padrão à lista de padrões
    padroes.append(padrao)
  # Adicionar os padrões ao objeto Matcher com o nome da categoria e subcategoria
  matcher.add(nome, padroes)

# Criar uma lista vazia para armazenar os rótulos
rotulos = []

# Para cada mensagem no dataframe de mensagens
for j in range(len(df_mensagens)):
  # Obter a mensagem
  mensagem = df_mensagens.iloc[j]["mensagem"]
  # Processar a mensagem usando o modelo do spaCy
  doc = nlp(mensagem)
  # Aplicar o objeto Matcher ao documento processado
  matches = matcher(doc)
  # Se houver algum resultado do Matcher
  if matches:
    # Obter o primeiro resultado (o mais relevante)
    match_id, start, end = matches[0]
    # Obter o nome da categoria e subcategoria correspondente ao resultado
    rotulo = nlp.vocab.strings[match_id]
    # Adicionar o rótulo à lista de rótulos
    rotulos.append(rotulo)
  # Se não houver nenhum resultado do Matcher
  else:
    # Adicionar um rótulo vazio à lista de rótulos
    rotulos.append("")

# Adicionar a lista de rótulos como uma nova coluna no dataframe de mensagens
df_mensagens["rotulo"] = rotulos

# Salvar o dataframe como um novo arquivo csv
df_mensagens.to_csv("mensagens_rotuladas.csv", index=False)


In [ ]:
# Importar as bibliotecas
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Ler os dataframes
df_mensagens = pd.read_csv("mensagens.csv")
df_amostra = pd.read_csv("amostra_rotulada.csv")

# Separar as mensagens e os rótulos da amostra
X_train = df_amostra["mensagem"]
y_train = df_amostra["rotulo"]

# Separar as mensagens não rotuladas
X_test = df_mensagens[df_mensagens["rotulo"].isnull()]["mensagem"]

# Transformar as mensagens em vetores numéricos usando TF-IDF
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Treinar um modelo de classificação usando Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

# Usar o modelo para rotular as mensagens não rotuladas, calculando a probabilidade de cada categoria
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

# Criar um dataframe com as mensagens não rotuladas, os rótulos preditos e as probabilidades
df_resultado = pd.DataFrame({"mensagem": X_test, "rotulo": y_pred, "probabilidade": y_prob.max(axis=1)})

# Salvar o dataframe como um novo arquivo csv
df_resultado.to_csv("resultado.csv", index=False)

In [ ]:
# Importar as bibliotecas
import pandas as pd
import matplotlib.pyplot as plt

# Ler o dataframe
df_mensagens = pd.read_csv("mensagens_rotuladas.csv")

# Agrupar as mensagens por rótulo e contar a frequência de cada rótulo
df_frequencia = df_mensagens.groupby("rotulo").count()

# Criar um gráfico de barras horizontal com os rótulos e as frequências
plt.barh(df_frequencia.index, df_frequencia["mensagem"])
# Adicionar um título ao gráfico
plt.title("Frequência dos rótulos das mensagens")
# Adicionar um rótulo ao eixo x
plt.xlabel("Frequência")
# Adicionar um rótulo ao eixo y
plt.ylabel("Rótulo")
# Mostrar o gráfico
plt.show()

In [ ]:
# Importar as bibliotecas
import pandas as pd
import spacy
import seaborn as sns

# Carregar o modelo de português do spaCy
nlp = spacy.load("pt_core_news_sm")

# Ler o dataframe
df_mensagens = pd.read_csv("mensagens_rotuladas.csv")

# Criar uma lista vazia para armazenar as palavras e os rótulos
palavras = []
rotulos = []

# Para cada mensagem e rótulo no dataframe
for i in range(len(df_mensagens)):
  # Obter a mensagem e o rótulo
  mensagem = df_mensagens.iloc[i]["mensagem"]
  rotulo = df_mensagens.iloc[i]["rotulo"]
  # Processar a mensagem usando o modelo do spaCy
  doc = nlp(mensagem)
  # Para cada token no documento processado
  for token in doc:
    # Se o token for alfanumérico, não for pontuação e não for uma palavra vazia (stop word)
    if token.is_alpha and not token.is_punct and not token.is_stop:
      # Adicionar a forma lematizada do token à lista de palavras
      palavras.append(token.lemma_)
      # Adicionar o rótulo correspondente à lista de rótulos
      rotulos.append(rotulo)

# Criar um dataframe com as palavras e os rótulos
df_palavras = pd.DataFrame({"palavra": palavras, "rotulo": rotulos})

# Agrupar as palavras por rótulo e contar a frequência de cada palavra
df_frequencia = df_palavras.groupby(["rotulo", "palavra"]).count()

# Ordenar as frequências em ordem decrescente
df_frequencia = df_frequencia.sort_values(by="rotulo", ascending=False)

# Criar um gráfico de barras horizontal com os rótulos, as palavras e as frequências
sns.barplot(x="rotulo", y="palavra", data=df_frequencia)
# Adicionar um título ao gráfico
plt.title("Frequência das palavras por rótulo")
# Adicionar um rótulo ao eixo x
plt.xlabel("Rótulo")
# Adicionar um rótulo ao eixo y
plt.ylabel("Palavra")
# Mostrar o gráfico
plt.show()

In [ ]:
# Importar as bibliotecas
import pandas as pd
import nltk
import matplotlib.pyplot as plt

# Baixar o pacote punkt do nltk
nltk.download("punkt")

# Ler o dataframe
df_mensagens = pd.read_csv("mensagens.csv")

# Criar uma lista vazia para armazenar os conjuntos de frases + palavras
conjuntos = []

# Para cada mensagem no dataframe
for i in range(len(df_mensagens)):
  # Obter a mensagem
  mensagem = df_mensagens.iloc[i]["mensagem"]
  # Tokenizar a mensagem em sentenças usando o nltk
  sentencas = nltk.sent_tokenize(mensagem)
  # Para cada sentença na mensagem
  for sentenca in sentencas:
    # Tokenizar a sentença em palavras usando o nltk
    palavras = nltk.word_tokenize(sentenca)
    # Se a sentença tiver pelo menos duas palavras
    if len(palavras) >= 2:
      # Obter a primeira e a última palavra da sentença
      primeira = palavras[0]
      ultima = palavras[-1]
      # Criar um conjunto com a frase formada pela primeira palavra e um espaço e a última palavra
      conjunto = (primeira + " ", ultima)
      # Adicionar o conjunto à lista de conjuntos
      conjuntos.append(conjunto)

# Criar um dataframe com os conjuntos e contar a frequência de cada conjunto
df_frequencia = pd.DataFrame(conjuntos, columns=["frase", "palavra"]).value_counts()

# Ordenar as frequências em ordem decrescente
df_frequencia = df_frequencia.sort_values(ascending=False)

# Criar um gráfico de barras horizontal com as frases, as palavras e as frequências
plt.barh(df_frequencia.index, df_frequencia.values)
# Adicionar um título ao gráfico
plt.title("Frequência dos conjuntos de frases + palavras")
# Adicionar um rótulo ao eixo x
plt.xlabel("Frequência")
# Adicionar um rótulo ao eixo y
plt.ylabel("Conjunto")
# Mostrar o gráfico
plt.show()
